<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Simulated-Data" data-toc-modified-id="Simulated-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Simulated Data</a></span><ul class="toc-item"><li><span><a href="#Normal-Distribution" data-toc-modified-id="Normal-Distribution-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Normal Distribution</a></span></li></ul></li><li><span><a href="#Real-world-Data" data-toc-modified-id="Real-world-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Real-world Data</a></span><ul class="toc-item"><li><span><a href="#AAPL" data-toc-modified-id="AAPL-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>AAPL</a></span></li><li><span><a href="#IBM" data-toc-modified-id="IBM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>IBM</a></span></li><li><span><a href="#MSFT" data-toc-modified-id="MSFT-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>MSFT</a></span></li><li><span><a href="#WMT" data-toc-modified-id="WMT-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>WMT</a></span></li></ul></li></ul></div>

# VaR methods

In [1]:
import numpy as np, pandas as pd, scipy as sp
from math import *
from scipy import stats
from scipy.stats import norm
import statsmodels.api as sm
from scipy import stats,random
import pandas_datareader.data as web
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="darkgrid")
p = print

## Simulated Data

In [2]:
np.random.seed(12345)

In [3]:
n_simulations = 5000000

### Normal Distribution

In [4]:
random_r = stats.norm.rvs(loc=0, scale=1.0, size=n_simulations)
random_r

array([-0.20470766,  0.47894334, -0.51943872, ..., -0.54125524,
        0.627719  ,  1.85312346])

In [5]:
def Parametric_VaR(CL, r):
    mean = np.mean(r)
    std_dev = np.std(r)
    
    VaR = norm.ppf(1-CL, mean, std_dev)
    
    return VaR

In [6]:
VaR_90 = Parametric_VaR(0.90, random_r)
VaR_95 = Parametric_VaR(0.95, random_r)
VaR_99 = Parametric_VaR(0.99, random_r)

In [7]:
p('VaR_90:', VaR_90)
p('VaR_95:', VaR_95)
p('VaR_99:', VaR_99)

VaR_90: -1.280980754610046
VaR_95: -1.644222635262718
VaR_99: -2.3256039932657604


In [8]:
def Modified_VaR(CL, r):
    std_dev = np.std(r)
    S = stats.skew(r)
    K = stats.kurtosis(r)
    
    z = norm.ppf(1-CL)
    t = z+1/6.*(z**2-1)*S+1/24.*(z**3-3*z)*K-1/36.*(2*z**3-5*z)*S**2
    
    mVaR = t * std_dev
    
    return mVaR

In [9]:
mVaR_90 = Modified_VaR(0.90, random_r)
mVaR_95 = Modified_VaR(0.95, random_r)
mVaR_99 = Modified_VaR(0.99, random_r)

In [10]:
p('mVaR_90:', mVaR_90)
p('mVaR_95:', mVaR_95)
p('mVaR_99:', mVaR_99)

mVaR_90: -1.2814669339667255
mVaR_95: -1.6447198423705713
mVaR_99: -2.32598886620861


In [11]:
def Historical_Simulation_VaR(CL, r):
    sorted_r = np.sort(r)
    
    SL = (1-CL) * 100
    
    hsVaR = np.percentile(sorted_r, SL)
    
    return hsVaR

In [12]:
HS_VaR_90 = Historical_Simulation_VaR(0.90, random_r)
HS_VaR_95 = Historical_Simulation_VaR(0.95, random_r)
HS_VaR_99 = Historical_Simulation_VaR(0.99, random_r)

In [13]:
p('HS_VaR_90:', HS_VaR_90)
p('HS_VaR_95:', HS_VaR_95)
p('HS_VaR_99:', HS_VaR_99)

HS_VaR_90: -1.2801096875771976
HS_VaR_95: -1.6442959069281922
HS_VaR_99: -2.3268461343672713


In [14]:
def Monte_Carlo_Simulation_VaR(CL, r, n):
    mean = np.mean(r)
    std_dev = np.std(r)
    
    sp.random.seed(12345)
    norm_r = np.random.normal(mean, std_dev, n)
    
    sorted_r = np.sort(norm_r)
    
    SL = (1-CL) * 100
    mcsVaR = np.percentile(sorted_r, SL)

    return mcsVaR

In [15]:
MCS_VaR_90 = Monte_Carlo_Simulation_VaR(0.90, random_r, n_simulations)
MCS_VaR_95 = Monte_Carlo_Simulation_VaR(0.95, random_r, n_simulations)
MCS_VaR_99 = Monte_Carlo_Simulation_VaR(0.99, random_r, n_simulations)

In [16]:
p('MCS_VaR_90:', MCS_VaR_90)
p('MCS_VaR_95:', MCS_VaR_95)
p('MCS_VaR_99:', MCS_VaR_99)

MCS_VaR_90: -1.279539115488807
MCS_VaR_95: -1.6436650076254034
MCS_VaR_99: -2.3261021710556857


In [17]:
start_date, end_date = '01/01/2007', '10/18/2019'

In [18]:
source = 'yahoo'

In [19]:
# # Get the stock prices
# AAPL = web.DataReader('AAPL', source, start_date, end_date)
# IBM = web.DataReader('IBM', source, start_date, end_date)
# MSFT = web.DataReader('MSFT', source, start_date, end_date)
# WMT = web.DataReader('WMT', source, start_date, end_date)

In [20]:
# # Save prices to a file
# AAPL.to_csv('AAPL.csv')
# IBM.to_csv('IBM.csv')
# MSFT.to_csv('MSFT.csv')
# WMT.to_csv('WMT.csv')

## Real-world Data

### AAPL

In [21]:
AAPL = pd.read_csv('AAPL.csv', index_col='Date', parse_dates=True)

In [22]:
AAPL_returns = AAPL['Close'].pct_change().fillna(0)

In [23]:
position = AAPL['Close'][-1]

In [24]:
n_shares = 1000

In [25]:
VaR_99 = Parametric_VaR(0.99, AAPL_returns)
mVaR_99 = Modified_VaR(0.99, AAPL_returns)
HS_VaR_99 = Historical_Simulation_VaR(0.99, AAPL_returns)
MCS_VaR_99 = Monte_Carlo_Simulation_VaR(0.99, AAPL_returns, n_simulations)

In [26]:
p('Holding:', position * n_shares)
p('VaR_99:', position * VaR_99)
p('mVaR_99:', position * mVaR_99)
p('HS_VaR_99:', position * HS_VaR_99)
p('MCS_VaR_99:', position * MCS_VaR_99)

Holding: 236410.00366210938
VaR_99: -10.602120145248055
mVaR_99: -18.32247044080731
HS_VaR_99: -13.208256795930666
MCS_VaR_99: -10.604447740582202


### IBM

In [27]:
IBM = pd.read_csv('IBM.csv', index_col='Date', parse_dates=True)

In [28]:
IBM_returns = IBM['Close'].pct_change().fillna(0)

In [29]:
n_shares = 1000

In [30]:
position = IBM['Close'][-1]

In [31]:
VaR_99 = Parametric_VaR(0.99, IBM_returns)
mVaR_99 = Modified_VaR(0.99, IBM_returns)
HS_VaR_99 = Historical_Simulation_VaR(0.99, IBM_returns)
MCS_VaR_99 = Monte_Carlo_Simulation_VaR(0.99, IBM_returns, n_simulations)

In [32]:
p('Holding:', position * n_shares)
p('VaR_99:', position * VaR_99)
p('mVaR_99:', position * mVaR_99)
p('HS_VaR_99:', position * HS_VaR_99)
p('MCS_VaR_99:', position * MCS_VaR_99)

Holding: 134089.99633789062
VaR_99: -4.344007548836183
mVaR_99: -7.121694246849648
HS_VaR_99: -5.620374673043631
MCS_VaR_99: -4.34494363763043


### MSFT

In [33]:
MSFT = pd.read_csv('MSFT.csv', index_col='Date', parse_dates=True)

In [34]:
MSFT_returns = MSFT['Close'].pct_change().fillna(0)

In [35]:
n_shares = 1000

In [36]:
position = MSFT['Close'][-1]

In [37]:
VaR_99 = Parametric_VaR(0.99, MSFT_returns)
mVaR_99 = Modified_VaR(0.99, MSFT_returns)
HS_VaR_99 = Historical_Simulation_VaR(0.99, MSFT_returns)
MCS_VaR_99 = Monte_Carlo_Simulation_VaR(0.99, MSFT_returns, n_simulations)

In [38]:
p('Holding:', position * n_shares)
p('VaR_99:', position * VaR_99)
p('mVaR_99:', position * mVaR_99)
p('HS_VaR_99:', position * HS_VaR_99)
p('MCS_VaR_99:', position * MCS_VaR_99)

Holding: 137410.00366210938
VaR_99: -5.377986009481361
mVaR_99: -10.116270850392512
HS_VaR_99: -6.302199279887439
MCS_VaR_99: -5.379156101190403


### WMT

In [39]:
WMT = pd.read_csv('WMT.csv', index_col='Date', parse_dates=True)

In [40]:
WMT_returns = WMT['Close'].pct_change().fillna(0)

In [41]:
n_shares = 1000

In [42]:
position = WMT['Close'][-1]

In [43]:
VaR_99 = Parametric_VaR(0.99, WMT_returns)
mVaR_99 = Modified_VaR(0.99, WMT_returns)
HS_VaR_99 = Historical_Simulation_VaR(0.99, WMT_returns)
MCS_VaR_99 = Monte_Carlo_Simulation_VaR(0.99, WMT_returns, n_simulations)

In [44]:
p('Holding:', position * n_shares)
p('VaR_99:', position * VaR_99)
p('mVaR_99:', position * mVaR_99)
p('HS_VaR_99:', position * HS_VaR_99)
p('MCS_VaR_99:', position * MCS_VaR_99)

Holding: 119139.99938964844
VaR_99: -3.405036524732503
mVaR_99: -7.2136966193783065
HS_VaR_99: -3.949656621020377
MCS_VaR_99: -3.4057750669833786
